In [1]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [2]:
list=[["""{'id': 1, 'name': 'Slatan', 'surname': 'Yav', 'age': 24}"""],
["""{'id': 2, 'name': 'Nikos', 'surname': 'Stef', 'age': 34}"""],
["""{'id': 3, 'name': 'Panos', 'surname': 'Rodes', 'age': 44}"""]]

df=spark.createDataFrame(list,['Body_decoded'])


df.show(truncate=False)
df.printSchema()

+---------------------------------------------------------+
Body_decoded |
+---------------------------------------------------------+
{'id': 1, 'name': 'Slatan', 'surname': 'Yav', 'age': 24} |
{'id': 2, 'name': 'Nikos', 'surname': 'Stef', 'age': 34} |
{'id': 3, 'name': 'Panos', 'surname': 'Rodes', 'age': 44}|
+---------------------------------------------------------+

root
-- Body_decoded: string (nullable = true)

In [3]:
df.show() #sample dataframe

#+---------------------------------------------------------+
#|Body_decoded                                             |
#+---------------------------------------------------------+
#|{'id': 1, 'name': 'Slatan', 'surname': 'Yav', 'age': 24} |
#|{'id': 2, 'name': 'Nikos', 'surname': 'Stef', 'age': 34} |
#|{'id': 3, 'name': 'Panos', 'surname': 'Rodes', 'age': 44}|
#+---------------------------------------------------------+
#root
 #|-- Body_decoded: string (nullable = true) #schema


from pyspark.sql.types import *
from pyspark.sql import functions as F

schema=MapType(StringType(),StringType())
columns_selected=['name','surname']

df.withColumn("Body_decoded", F.from_json(F.regexp_replace("Body_decoded",'[^"\d,]?(\d+)',"'$1'")\
                                              ,schema))\
  .select(*["Body_decoded.{}".format(x) for x in columns_selected]).printSchema()

#+------+-------+
#|name  |surname|
#+------+-------+
#|Slatan|Yav    |
#|Nikos |Stef   |
#|Panos |Rodes  |
#+------+-------+



+--------------------+
 Body_decoded|
+--------------------+
{'id': 1, 'name':...|
{'id': 2, 'name':...|
{'id': 3, 'name':...|
+--------------------+

root
-- name: string (nullable = true)
-- surname: string (nullable = true)

In [4]:
schema = StructType(
    [
      StructField('name', StringType(), True),
      StructField('surname', IntegerType(), True)
    ]
)

from pyspark.sql.types import *
from pyspark.sql import functions as F

#schema=MapType(StringType(),StringType())

df.withColumn("Body_decoded", F.from_json(F.regexp_replace("Body_decoded",'[^"\d,]?(\d+)',"'$1'")\
                                              ,schema))\
  .show()

+------------+
Body_decoded|
+------------+
 null|
 null|
 null|
+------------+

In [6]:
list=[['John'    ,  'PIPE'],
      ['Hema'     , 'BALL-KG'],
      ['Basha'     ,'BALL-KG'],
      ['Hari'      ,'BALL'],
      ['Bijju'     ,'BAG']]

df=spark.createDataFrame(list,['Name','Product'])

df.show()

+-----+-------+
 Name|Product|
+-----+-------+
 John| PIPE|
 Hema|BALL-KG|
Basha|BALL-KG|
 Hari| BALL|
Bijju| BAG|
+-----+-------+

In [7]:
df.withColumn("Product", F.when(F.col("Product")=="BALL-KG",F.lit("BALL")).otherwise(F.col("Product")))

+-----+-------+
 Name|Product|
+-----+-------+
 John| PIPE|
 Hema| BALL|
Basha| BALL|
 Hari| BALL|
Bijju| BAG|
+-----+-------+

In [8]:
list=[['2020-05-06 12:20:16','Danger Zone','iPhone11','16.17.17.17',-7,'usr1'],
      ['2020-05-06 12:28:46','Test Zone','iPhone11','16.17.17.17',0,'usr1'],
      ['2020-05-06 12:23:36','Test Zone','iPhone11','16.17.17.17',0,'usr1'],
      ['2020-05-06 12:23:36','Test Zone','Samsung','18.18.17.17',0,'usr3']]

df=spark.createDataFrame(list,['Col1','Col2','Col3','Col4','Col5','Col6'])

df.show()

+-------------------+-----------+--------+-----------+----+----+
 Col1| Col2| Col3| Col4|Col5|Col6|
+-------------------+-----------+--------+-----------+----+----+
2020-05-06 12:20:16|Danger Zone|iPhone11|16.17.17.17| -7|usr1|
2020-05-06 12:28:46| Test Zone|iPhone11|16.17.17.17| 0|usr1|
2020-05-06 12:23:36| Test Zone|iPhone11|16.17.17.17| 0|usr1|
2020-05-06 12:23:36| Test Zone| Samsung|18.18.17.17| 0|usr3|
+-------------------+-----------+--------+-----------+----+----+

In [9]:

w1=Window.partitionBy("Col6").orderBy(F.unix_timestamp("Col1","yyyy-MM-dd HH:mm:ss")).rangeBetween(-300, -1)
w2=Window.partitionBy("Col6").orderBy(F.unix_timestamp("Col1","yyyy-MM-dd HH:mm:ss")).rangeBetween(1, 300)
df.withColumn("collect1", F.collect_list(F.array(*["col2","col3","col4","col5"])).over(w1))\
.withColumn("collect2", F.collect_list(F.array(*["col2","col3","col4","col5"])).over(w2))\
.withColumn("collect1", F.when((F.size("collect1")==0) &\
(F.size("collect2")!=0),F.col("collect2")[0])\
.otherwise(F.col("collect1")[0])).drop("collect2")\
.filter("""collect1[0]!=col2 and collect1[1]=col3 and collect1[2]=col4\
 and (collect1[3]+7=col5 or collect1[3]-7=col5)""").drop("collect1")\
.show(truncate=False)

+-------------------+-----------+--------+-----------+----+----+
Col1 |Col2 |Col3 |Col4 |Col5|Col6|
+-------------------+-----------+--------+-----------+----+----+
2020-05-06 12:20:16|Danger Zone|iPhone11|16.17.17.17|-7 |usr1|
2020-05-06 12:23:36|Test Zone |iPhone11|16.17.17.17|0 |usr1|
+-------------------+-----------+--------+-----------+----+----+

In [10]:

#+-------------------+-----------+--------+-----------+----+----+
#|               Col1|       Col2|    Col3|       Col4|Col5|Col6|
#+-------------------+-----------+--------+-----------+----+----+
#|2020-05-06 12:20:16|Danger Zone|iPhone11|16.17.17.17|  -7|usr1|
#|2020-05-06 12:28:46|  Test Zone|iPhone11|16.17.17.17|   0|usr1|
#|2020-05-06 12:23:36|  Test Zone|iPhone11|16.17.17.17|   0|usr1|
#|2020-05-06 12:23:36|  Test Zone| Samsung|18.18.17.17|   0|usr3|
#+-------------------+-----------+--------+-----------+----+----+
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w1=Window.partitionBy("Col6").orderBy(F.unix_timestamp("Col1","yyyy-MM-dd HH:mm:ss")).rangeBetween(-300, -1)
w2=Window.partitionBy("Col6").orderBy(F.unix_timestamp("Col1","yyyy-MM-dd HH:mm:ss")).rangeBetween(1, 300)
df.withColumn("collect1", F.collect_list(F.array(*["col2","col3","col4","col5"])).over(w1))\
  .withColumn("collect2", F.collect_list(F.array(*["col2","col3","col4","col5"])).over(w2))\
  .withColumn("collect1", F.when((F.size("collect1")==0) &\
                                 (F.size("collect2")!=0),F.col("collect2")[0])\
              .otherwise(F.col("collect1")[0])).drop("collect2")\
  .filter("""collect1[0]!=col2 and collect1[1]=col3 and collect1[2]=col4\
             and (collect1[3]+7=col5 or collect1[3]-7=col5)""").drop("collect1")\
  .show(truncate=False)


+-------------------+-----------+--------+-----------+----+----+
Col1 |Col2 |Col3 |Col4 |Col5|Col6|
+-------------------+-----------+--------+-----------+----+----+
2020-05-06 12:20:16|Danger Zone|iPhone11|16.17.17.17|-7 |usr1|
2020-05-06 12:23:36|Test Zone |iPhone11|16.17.17.17|0 |usr1|
+-------------------+-----------+--------+-----------+----+----+

In [11]:

#+-------------------+-----------+--------+-----------+----+----+
#|               Col1|       Col2|    Col3|       Col4|Col5|Col6|
#+-------------------+-----------+--------+-----------+----+----+
#|2020-05-06 12:20:16|Danger Zone|iPhone11|16.17.17.17|  -7|usr1|
#|2020-05-06 12:28:46|  Test Zone|iPhone11|16.17.17.17|   0|usr1|
#|2020-05-06 12:23:36|  Test Zone|iPhone11|16.17.17.17|   0|usr1|
#|2020-05-06 12:23:36|  Test Zone| Samsung|18.18.17.17|   0|usr3|
#+-------------------+-----------+--------+-----------+----+----+
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w1=Window.partitionBy("Col6").orderBy(F.unix_timestamp("Col1","yyyy-MM-dd HH:mm:ss")).rangeBetween(-300, -1)
w2=Window.partitionBy("Col6").orderBy(F.unix_timestamp("Col1","yyyy-MM-dd HH:mm:ss")).rangeBetween(1, 300)
df.withColumn("collect1", F.collect_list(F.array(*["col2","col3","col4","col5"])).over(w1))\
  .withColumn("collect2", F.collect_list(F.array(*["col2","col3","col4","col5"])).over(w2))\
  .withColumn("collect1", F.when((F.size("collect1")==0) &\
                                 (F.size("collect2")!=0),F.col("collect2")).otherwise(F.col("collect1"))).drop("collect2")\
  .filter("""exists(collect1,x-> x[0]!=col2 and x[1]==col3 and x[2]==col4 and (x[3]+7=col5 or x[3]-7=col5)=True)""")\
  .drop("collect1").show(truncate=False)

In [12]:
.filter("""exists(collect1,x-> x[0]!=col2 and x[1]==col3 and x[2]==col4 and (x[3]+7=col5 or x[3]-7=col5)=True)""")\

In [13]:
w1=Window.partitionBy("Col6").orderBy(F.unix_timestamp("Col1","yyyy-MM-dd HH:mm:ss")).rangeBetween(-300, -1)
w2=Window.partitionBy("Col6").orderBy(F.unix_timestamp("Col1","yyyy-MM-dd HH:mm:ss")).rangeBetween(1, 300)
df.withColumn("collect1", F.collect_list(F.array(*["col2","col3","col4","col5"])).over(w1))\
.withColumn("collect2", F.collect_list(F.array(*["col2","col3","col4","col5"])).over(w2))\
.withColumn("collect1", F.when((F.size("collect1")==0) &\
(F.size("collect2")!=0),F.col("collect2")).otherwise(F.col("collect1"))).drop("collect2")\
.filter("""exists(collect1,x-> x[0]!=col2 and x[1]==col3 and x[2]==col4 and (x[3]+7=col5 or x[3]-7=col5)=True)""")\
.drop("collect1").show(truncate=False)

+-------------------+-----------+--------+-----------+----+----+
Col1 |Col2 |Col3 |Col4 |Col5|Col6|
+-------------------+-----------+--------+-----------+----+----+
2020-05-06 12:20:16|Danger Zone|iPhone11|16.17.17.17|-7 |usr1|
2020-05-06 12:23:36|Test Zone |iPhone11|16.17.17.17|0 |usr1|
+-------------------+-----------+--------+-----------+----+----+

In [14]:
.filter("""exists(collect1,x-> x[0]!=col2 and x[1]==col3 and x[2]==col4 and (x[3]+7=col5 or x[3]-7=col5)=True)""")\

In [15]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w1=Window.partitionBy("Col6").orderBy(F.unix_timestamp("Col1","yyyy-MM-dd HH:mm:ss")).rangeBetween(-300, 300)
df.withColumn("collect", F.collect_list(F.struct(*[F.col(x).alias(x) for x in ['col3','col4','col5','col2']])).over(w1))\
  .withColumn("collect1", F.collect_list("col5").over(w1))\
  .filter("""exists(collect,x-> x.col3==col3 and x.col4==col4)""")\
  .filter("""exists(collect1,x->x=-7)=True and exists(collect1,x->x=0)=True""").drop("collect","collect1")\
  .show(truncate=False)

+-------------------+-----------+--------+-----------+----+----+
Col1 |Col2 |Col3 |Col4 |Col5|Col6|
+-------------------+-----------+--------+-----------+----+----+
2020-05-06 12:20:16|Danger Zone|iPhone11|16.17.17.17|-7 |usr1|
2020-05-06 12:23:36|Test Zone |iPhone11|16.17.17.17|0 |usr1|
+-------------------+-----------+--------+-----------+----+----+

In [16]:
#So if you look at the data only Row1 and Row3 satisfy the conditions where the logons happen within 5 mins of duration and by the same user where col5 changes from -7 to 0.All the Col3 and Col4 value remains same within 5 minutes of duration.Even when the value is -7 the Col2 value is Danger Zone in Row1

In [17]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w1=Window.partitionBy("Col6").orderBy(F.unix_timestamp("Col1","yyyy-MM-dd HH:mm:ss")).rangeBetween(-300, 300)
df.withColumn("collect", F.collect_list(F.array(*[F.col(x) for x in ['col3','col4','col5','col2']])).over(w1))\
  .show(truncate=False)

+-------------------+-----------+--------+-----------+----+----+---------------------------------------------------------------------------------+
Col1 |Col2 |Col3 |Col4 |Col5|Col6|collect |
+-------------------+-----------+--------+-----------+----+----+---------------------------------------------------------------------------------+
2020-05-06 12:23:36|Test Zone |Samsung |18.18.17.17|0 |usr3|[[Samsung, 18.18.17.17, 0, Test Zone]] |
2020-05-06 12:20:16|Danger Zone|iPhone11|16.17.17.17|-7 |usr1|[[iPhone11, 16.17.17.17, -7, Danger Zone], [iPhone11, 16.17.17.18, 0, Test Zone]]|
2020-05-06 12:23:36|Test Zone |iPhone11|16.17.17.18|0 |usr1|[[iPhone11, 16.17.17.17, -7, Danger Zone], [iPhone11, 16.17.17.18, 0, Test Zone]]|
2020-05-06 12:28:46|Test Zone |iPhone11|16.17.17.17|0 |usr1|[[iPhone11, 16.17.17.17, 0, Test Zone]] |
+-------------------+-----------+--------+-----------+----+----+---------------------------------------------------------------------------------+

In [18]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w1=Window.partitionBy("Col6").orderBy(F.unix_timestamp("Col1","yyyy-MM-dd HH:mm:ss")).rangeBetween(-300, 300)
df.withColumn("collect", F.collect_list(F.array(F.struct(*[F.col(x).alias(x) for x in ['col3','col4','col5','col2']]))).over(w1))\
  .printSchema()

root
-- Col1: string (nullable = true)
-- Col2: string (nullable = true)
-- Col3: string (nullable = true)
-- Col4: string (nullable = true)
-- Col5: long (nullable = true)
-- Col6: string (nullable = true)
-- collect: array (nullable = true)
 |-- element: array (containsNull = true)
 | |-- element: struct (containsNull = false)
 | | |-- col3: string (nullable = true)
 | | |-- col4: string (nullable = true)
 | | |-- col5: long (nullable = true)
 | | |-- col2: string (nullable = true)

In [19]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w1=Window.partitionBy("Col6").orderBy(F.unix_timestamp("Col1","yyyy-MM-dd HH:mm:ss")).rangeBetween(-300, 300)
df.withColumn("collect", F.collect_list(F.array(*[F.col(x).alias(x) for x in ['col3','col4','col5','col2']])).over(w1))\
  .show(truncate=False)

+-------------------+-----------+--------+-----------+----+----+---------------------------------------------------------------------------------+
Col1 |Col2 |Col3 |Col4 |Col5|Col6|collect |
+-------------------+-----------+--------+-----------+----+----+---------------------------------------------------------------------------------+
2020-05-06 12:23:36|Test Zone |Samsung |18.18.17.17|0 |usr3|[[Samsung, 18.18.17.17, 0, Test Zone]] |
2020-05-06 12:20:16|Danger Zone|iPhone11|16.17.17.17|-7 |usr1|[[iPhone11, 16.17.17.17, -7, Danger Zone], [iPhone11, 16.17.17.17, 0, Test Zone]]|
2020-05-06 12:23:36|Test Zone |iPhone11|16.17.17.17|0 |usr1|[[iPhone11, 16.17.17.17, -7, Danger Zone], [iPhone11, 16.17.17.17, 0, Test Zone]]|
2020-05-06 12:28:46|Test Zone |iPhone11|16.17.17.17|0 |usr1|[[iPhone11, 16.17.17.17, 0, Test Zone]] |
+-------------------+-----------+--------+-----------+----+----+---------------------------------------------------------------------------------+

In [20]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w1=Window.partitionBy("Col6").orderBy(F.unix_timestamp("Col1","yyyy-MM-dd HH:mm:ss")).rangeBetween(-300, 300)

df.withColumn("collect", F.collect_list("col5").over(w1))\
  .filter("""exists(collect,x->x=-7)=True and exists(collect,x->x=0)=True""").show()

+-------------------+-----------+--------+-----------+----+----+-------+
 Col1| Col2| Col3| Col4|Col5|Col6|collect|
+-------------------+-----------+--------+-----------+----+----+-------+
2020-05-06 12:20:16|Danger Zone|iPhone11|16.17.17.17| -7|usr1|[-7, 0]|
2020-05-06 12:23:36| Test Zone| Phone11|16.17.17.17| 0|usr1|[-7, 0]|
+-------------------+-----------+--------+-----------+----+----+-------+

In [21]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w1=Window.partitionBy("Col6").orderBy(F.unix_timestamp("Col1","yyyy-MM-dd HH:mm:ss")).rangeBetween(-300, 300)

df.withColumn("collect", F.collect_list("col5").over(w1))\
  .filter("""array_contains(collect,-7)=True and array_contains(collect,0)=True""").drop("collect").show()

+-------------------+-----------+--------+-----------+----+----+
 Col1| Col2| Col3| Col4|Col5|Col6|
+-------------------+-----------+--------+-----------+----+----+
2020-05-06 12:20:16|Danger Zone|iPhone11|16.17.17.17| -7|usr1|
2020-05-06 12:23:36| Test Zone| Phone11|16.17.17.17| 0|usr1|
+-------------------+-----------+--------+-----------+----+----+

In [22]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w1=Window.partitionBy("Col6").orderBy(F.unix_timestamp("Col1","yyyy-MM-dd HH:mm:ss")).rangeBetween(-300, 300)

df.withColumn("collect", F.collect_list("col5").over(w1))\
  .filter("""size(filter(collect,x->x!=-7 and x!=0))=size(collect)-2""").drop("collect").show()

+-------------------+-----------+--------+-----------+----+----+
 Col1| Col2| Col3| Col4|Col5|Col6|
+-------------------+-----------+--------+-----------+----+----+
2020-05-06 12:20:16|Danger Zone|iPhone11|16.17.17.17| -7|usr1|
2020-05-06 12:23:36| Test Zone| Phone11|16.17.17.17| 0|usr1|
+-------------------+-----------+--------+-----------+----+----+

In [23]:
Col1,Col2,Col3,Col4,Col5,Col6
2020-05-06 12:20:16,Danger Zone,iPhone11,16.17.17.17,-7,usr1-----Row1
2020-05-06 12:23:36,Test Zone,Phone11,16.17.17.17,0,us1----------Row3

In [24]:
list=[['2019-09-17',         22],
      ['2019-09-17',          11],
      ['2019-09-17',           9]]

df=spark.createDataFrame(list,['start_date','daypart_hour'])

df.show()

+----------+------------+
start_date|daypart_hour|
+----------+------------+
2019-09-17| 22|
2019-09-17| 11|
2019-09-17| 9|
+----------+------------+

In [25]:
df.withColumn("start_dt_ts", F.to_timestamp(F.concat("start_date","daypart_hour"),"yyyy-MM-ddHH")).show()

+----------+------------+-------------------+
start_date|daypart_hour| start_dt_ts|
+----------+------------+-------------------+
2019-09-17| 22|2019-09-17 22:00:00|
2019-09-17| 11|2019-09-17 11:00:00|
2019-09-17| 9|2019-09-17 09:00:00|
+----------+------------+-------------------+

In [26]:
df.withColumn("start_dt_ts", \
              F.date_format(F.to_timestamp(F.concat("start_date","daypart_hour"),"yyyy-MM-ddHH"),\
                                          "yyyy-MM-dd hh:ss:SSa")).show(truncate=False)

+----------+------------+---------------------+
start_date|daypart_hour|start_dt_ts |
+----------+------------+---------------------+
2019-09-17|22 |2019-09-17 10:00:00PM|
2019-09-17|11 |2019-09-17 11:00:00AM|
2019-09-17|9 |2019-09-17 09:00:00AM|
+----------+------------+---------------------+

In [27]:
df1 = spark.createDataFrame(
     [('ll',5),
     ('yy',6)],
     ('x','days'))

df = spark.createDataFrame(
        [
    ('ll','2020-01-05','1','10'),
    ('ll','2020-01-06','1','10'),
    ('ll','2020-01-07','1','10'),
    ('ll','2020-01-08','1','10'),
    ('ll','2020-01-09','1','10'),
    ('ll','2020-01-10','1','10'),
    ('ll','2020-01-11','1','20'),
    ('ll','2020-01-12','1','10'),
    ('ll','2020-01-05','2','30'),
    ('ll','2020-01-06','2','30'),
    ('ll','2020-01-07','2','30'),
    ('ll','2020-01-08','2','40'),
    ('ll','2020-01-09','2','30'),
    ('ll','2020-01-10','2','10'),
    ('ll','2020-01-11','2','10'),
    ('ll','2020-01-12','2','10'),
    ('yy','2020-01-05','1','20'),
    ('yy','2020-01-06','1','20'),
    ('yy','2020-01-07','1','20'),
    ('yy','2020-01-08','1','20'),
    ('yy','2020-01-09','1','20'),
    ('yy','2020-01-10','1','40'),
    ('yy','2020-01-11','1','20'),
    ('yy','2020-01-12','1','20'),
    ('yy','2020-01-05','2','40'),
    ('yy','2020-01-06','2','40'),
    ('yy','2020-01-07','2','40'),
    ('yy','2020-01-08','2','40'),
    ('yy','2020-01-09','2','40'),
    ('yy','2020-01-10','2','40'),
    ('yy','2020-01-11','2','60'),
    ('yy','2020-01-12','2','40')],
        ('x','date','flag','value'))

In [28]:
df1.show()
df.show()

+---+----+
 x|days|
+---+----+
 ll| 5|
 yy| 6|
+---+----+

+---+----------+----+-----+
 x| date|flag|value|
+---+----------+----+-----+
 ll|2020-01-05| 1| 10|
 ll|2020-01-06| 1| 10|
 ll|2020-01-07| 1| 10|
 ll|2020-01-08| 1| 10|
 ll|2020-01-09| 1| 10|
 ll|2020-01-10| 1| 10|
 ll|2020-01-11| 1| 20|
 ll|2020-01-12| 1| 10|
 ll|2020-01-05| 2| 30|
 ll|2020-01-06| 2| 30|
 ll|2020-01-07| 2| 30|
 ll|2020-01-08| 2| 40|
 ll|2020-01-09| 2| 30|
 ll|2020-01-10| 2| 10|
 ll|2020-01-11| 2| 10|
 ll|2020-01-12| 2| 10|
 yy|2020-01-05| 1| 20|
 yy|2020-01-06| 1| 20|
 yy|2020-01-07| 1| 20|
 yy|2020-01-08| 1| 20|
+---+----------+----+-----+
only showing top 20 rows

In [29]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window().partitionBy("x","flag")
w1=Window().partitionBy("x","flag").orderBy(F.to_date("date","yyyy-dd-MM"))

df.join(df1,['x'])\
  .withColumn("result", F.collect_list("value").over(w))\
  .withColumn("rowNum", F.row_number().over(w1)-1)\
  .withColumn("result", F.expr("""aggregate(transform(result,(x,i)->array(x,i)),0,(acc,x)-> \
                             IF((int(x[1])>=rowNum)and(int(x[1])<days+rowNum),int(x[0])+acc,acc))"""))\
  .drop("flag","rowNum","days").show()
 

+---+----------+-----+------+
 x| date|value|result|
+---+----------+-----+------+
 ll|2020-01-05| 10| 50|
 ll|2020-01-06| 10| 50|
 ll|2020-01-07| 10| 60|
 ll|2020-01-08| 10| 60|
 ll|2020-01-09| 10| 50|
 ll|2020-01-10| 10| 40|
 ll|2020-01-11| 20| 30|
 ll|2020-01-12| 10| 10|
 ll|2020-01-05| 30| 160|
 ll|2020-01-06| 30| 140|
 ll|2020-01-07| 30| 120|
 ll|2020-01-08| 40| 100|
 ll|2020-01-09| 30| 60|
 ll|2020-01-10| 10| 30|
 ll|2020-01-11| 10| 20|
 ll|2020-01-12| 10| 10|
 yy|2020-01-05| 20| 140|
 yy|2020-01-06| 20| 140|
 yy|2020-01-07| 20| 140|
 yy|2020-01-08| 20| 120|
 yy|2020-01-09| 20| 100|
 yy|2020-01-10| 40| 80|
 yy|2020-01-11| 20| 40|
 yy|2020-01-12| 20| 20|
 yy|2020-01-05| 40| 240|
 yy|2020-01-06| 40| 260|
 yy|2020-01-07| 40| 260|
 yy|2020-01-08| 40| 220|
 yy|2020-01-09| 40| 180|
 yy|2020-01-10| 40| 140|
 yy|2020-01-11| 60| 100|
 yy|2020-01-12| 40| 40|
+---+----------+-----+------+

In [30]:
    +---+----------+----+-----+------+
    |  x|      date|flag|value|result|
    +---+----------+----+-----+------+
    | ll|2020-01-05|   1|   10|    50|
    | ll|2020-01-06|   1|   10|    50|
    | ll|2020-01-07|   1|   10|    60|
    | ll|2020-01-08|   1|   10|    60|
    | ll|2020-01-09|   1|   10|    50|
    | ll|2020-01-10|   1|   10|    40|
    | ll|2020-01-11|   1|   20|    30|
    | ll|2020-01-12|   1|   10|    10|
    | ll|2020-01-05|   2|   30|   170|
    | ll|2020-01-06|   2|   30|   140|
    | ll|2020-01-07|   2|   30|   120|
    | ll|2020-01-08|   2|   40|   100|
    | ll|2020-01-09|   2|   30|    60|
    | ll|2020-01-10|   2|   10|    30|
    | ll|2020-01-11|   2|   10|    20|
    | ll|2020-01-12|   2|   10|    10|
    | yy|2020-01-05|   1|   20|   140|
    | yy|2020-01-06|   1|   20|   140|
    | yy|2020-01-07|   1|   20|   140|
    | yy|2020-01-08|   1|   20|   120|
    | yy|2020-01-09|   1|   20|   100|
    | yy|2020-01-10|   1|   40|    80|
    | yy|2020-01-11|   1|   20|    40|
    | yy|2020-01-12|   1|   20|    20|
    | yy|2020-01-05|   2|   40|   240|
    | yy|2020-01-06|   2|   40|   260|
    | yy|2020-01-07|   2|   40|   260|
    | yy|2020-01-08|   2|   40|   220|
    | yy|2020-01-09|   2|   40|   180|
    | yy|2020-01-10|   2|   40|   140|
    | yy|2020-01-11|   2|   60|   100|
    | yy|2020-01-12|   2|   40|    40|
    +---+----------+----+-----+------+

In [31]:
list=[[None,'val1','val2','val','A'],       
      ['val4',None,'val5','val','B'],       
      [None,None,'val7','val','C'],     
      [None,'val1','val8','val','A']]   

df=spark.createDataFrame(list,['col1','col2','col3','col4','event_type'])


df.show()


+----+----+----+----+----------+
col1|col2|col3|col4|event_type|
+----+----+----+----+----------+
null|val1|val2| val| A|
val4|null|val5| val| B|
null|null|val7| val| C|
null|val1|val8| val| A|
+----+----+----+----+----------+

In [32]:
df.show() #sampledata
#+----+----+----+----------+
#|col1|col2|col3|event_type|
#+----+----+----+----------+
#|null|val1|val2|         A|
#|val4|null|val5|         B|
#|null|null|val7|         C|
#|null|val1|val8|         A|
#+----+----+----+----------+


from pyspark.sql import functions as F

df.groupBy("event_type").agg\
(F.first(F.concat_ws(",",*[(F.when(F.col(x).isNotNull(), F.lit(x)))\
                           for x in df.columns if x!='event_type'])).alias("non_null_columns")).show()


#+----------+----------------+
#|event_type|non_null_columns|
#+----------+----------------+
#|         B|       col1,col3|
#|         C|            col3|
#|         A|       col2,col3|
#+----------+----------------+


+----+----+----+----+----------+
col1|col2|col3|col4|event_type|
+----+----+----+----+----------+
null|val1|val2| val| A|
val4|null|val5| val| B|
null|null|val7| val| C|
null|val1|val8| val| A|
+----+----+----+----+----------+

+----------+----------------+
event_type|non_null_columns|
+----------+----------------+
 B| col1,col3,col4|
 C| col3,col4|
 A| col2,col3,col4|
+----------+----------------+